# 3D Face recognition

This work is for face recognition / verification for 3D facial images. The file utilizes (at least on the current setting) LFW data base, which celebratires images database, we used it because we can get multiple images for the sane subject. The final Pipeline is as follows: 

Input raw image >> Face detection >> 3D reconstruction (3D rendered image) >> recognition / verification

The steps will be explained accordingly

## First part
### Dataset preparation

This part will mainly focus on downloading the database and filter it out. The filtration is done by the number of images in the each folder, currently, minimum of 7 images were selected, 3 images will be used for testing and the rest for reconstruction and training.

In [ ]:
# Downloading and extracting database
import os

dataset_dir = '/content/datasets/lfw'
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

os.chdir('/content/datasets/lfw')

!pwd
!wget http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz
!tar zxvf lfw-deepfunneled.tgz

In [ ]:
# Filtering the database based on the total number of  images in each subject  
!pwd # Just checking the path


import os
import shutil
import glob


num_imgs = 7
num_train = 5

root_path = './lfw-deepfunneled/*'
output_train_dir = './train'
output_test_dir = './test'

if not os.path.exists(output_train_dir):
    os.makedirs(output_train_dir)

if not os.path.exists(output_test_dir):
    os.makedirs(output_test_dir)

root_contents = glob.glob(root_path)

separator = os.path.sep

all_base_names = []
for this_dir in root_contents:
    if os.path.isdir(this_dir) and len(glob.glob(this_dir + '/*')) >= num_imgs:
      
      this_dir_base_name = this_dir.split(separator)[-1]
      all_base_names.append(this_dir_base_name)
      if not os.path.exists(os.path.join(output_train_dir, this_dir_base_name)):
          os.makedirs(os.path.join(output_train_dir, this_dir_base_name))
          
      if not os.path.exists(os.path.join(output_test_dir, this_dir_base_name)):
          os.makedirs(os.path.join(output_test_dir, this_dir_base_name))

      this_dir_contents = glob.glob(this_dir + '/*')
      for i, this_img in enumerate(this_dir_contents):
        # print(i, this_img)
        if i <= num_train:
          # print('This is training img', i, this_img)
          shutil.copy(this_img, os.path.join(output_train_dir, this_dir_base_name))
        if i > num_train and i <= num_imgs:
          # print('This is test img', i, this_img)
          shutil.copy(this_img, os.path.join(output_test_dir, this_dir_base_name))

Now, dataset preparation is done, we need to pass the data to the next stage, which is face detection

For the face detection, a model called DeepFace will be used, and will also be used in the latest recognition phase.

In [ ]:
# Prepare deepface repo and MTCNN
# DeepFace library can perform several tasks 
!pip install deepface
!pip install mtcnn


# print(this_dir)

### Face detection and face landmarks detection

For face detection, the final model I have utilized is the MTCNN model, it generate a JSON file with the bounding box and 5 points facial main landmarks, those landmarks are used subsequently for the 3D reconstruction of the face.


At this stage a number of images are selected to form the training database, just for testing purposes, the default setting in real world to include all of the subjects



In [ ]:
from mtcnn import MTCNN
import cv2
import os
import random


def landmarks_to_file(dir_path):

  detections_path = os.path.join(dir_path, 'detections')
  if not os.path.exists(detections_path):
    os.makedirs(detections_path)

  landmarks_detector = MTCNN()
  img_paths = glob.glob(dir_path + '/*.jpg')
  for img_path in img_paths:
    landmarks = landmarks_detector.detect_faces(cv2.imread(img_path, cv2.COLOR_BGR2RGB))
    img_base_name = img_path.split(os.path.sep)[-1].split('.')[0]
    if len(landmarks) > 0:
      with open(os.path.join(detections_path, img_base_name + '.txt'), 'w') as f:
        for value in landmarks[0]['keypoints']:
          f.write('{0} {1}\n'.format(landmarks[0]['keypoints'][value][0], 
                                  landmarks[0]['keypoints'][value][1]))

num_working_imgs = 10
random.shuffle(all_base_names)

database_train_root = '/content/datasets/lfw/train'
database_test_root = '/content/datasets/lfw/test'

select_subjects = []
for i in range(len(all_base_names)):
  if i <= num_working_imgs:
    select_subjects.append(all_base_names[i])
    
    this_train_dir = os.path.join(database_train_root, all_base_names[i])
    this_test_dir = os.path.join(database_test_root, all_base_names[i])

    landmarks_to_file(this_train_dir)
    landmarks_to_file(this_test_dir)

print('Selected images for database:')
for i in select_subjects:
  print(i)

Preparing GitHub repo and the depenceies for 3D part

In [ ]:
os.chdir('/content')
!git clone https://github.com/sicxu/Deep3DFaceRecon_pytorch.git
!pip install pillow argparse
!pip install kornia
!pip install dominate
!pip install trimesh
!pip install ninja

In [ ]:
import os 

face_recon_root = '/content/Deep3DFaceRecon_pytorch/'

# if not os.path.exists(face_recon_root):
#     os.makedirs(face_recon_root)
os.chdir(face_recon_root)

!git clone https://github.com/NVlabs/nvdiffrast


os.chdir('/content/Deep3DFaceRecon_pytorch/nvdiffrast')
!pip install .

In [ ]:
os.chdir('/content/Deep3DFaceRecon_pytorch/')
!git clone https://github.com/deepinsight/insightface.git
!cp -r ./insightface/recognition/arcface_torch ./models/

In [ ]:
# if not os.path.exists('/content/Deep3DFaceRecon_pytorch/BFM'):
#     os.makedirs('/content/Deep3DFaceRecon_pytorch/BFM')
os.chdir('/content/Deep3DFaceRecon_pytorch/BFM')


!wget https://www.dropbox.com/s/rzx3ajx8dcvft5m/morph_model.zip
!unzip morph_model.zip

!wget https://www.dropbox.com/s/isx5qs1emhjza38/Exp_Pca.zip
!unzip Exp_Pca.zip

In [ ]:
os.makedirs('/content/Deep3DFaceRecon_pytorch/checkpoints/face_3d_recon/')
os.chdir('/content/Deep3DFaceRecon_pytorch/checkpoints/face_3d_recon/')
!wget https://www.dropbox.com/s/wkp6oaceux7tah6/face_recon_feat0.2_augment-20221005T062920Z-001.zip
!unzip face_recon_feat0.2_augment-20221005T062920Z-001.zip

import shutil
shutil.move('./face_recon_feat0.2_augment/epoch_20.pth', '.')

In [ ]:
# Now, render the training data to create the database
os.chdir('/content/Deep3DFaceRecon_pytorch')

for i in select_subjects:
  dir_path = os.path.join(database_train_root, i)
  print('python test.py --name=face_3d_recon --epoch=20 --img_folder={0}'.format(dir_path))
  os.system('python test.py --name=face_3d_recon --epoch=20 --img_folder={0}'.format(dir_path))

In [ ]:
import glob
import os
import cv2
import matplotlib.pyplot as plt

def crop_and_copy(dist_dir, in_str):
  
  if not os.path.exists(dist_dir):
    os.makedirs(dist_dir)

  os.chdir('/content/Deep3DFaceRecon_pytorch/checkpoints/face_3d_recon/')
  all_images = glob.glob('./results/{0}/*/*.png'.format(in_str))
  

  if in_str == 'test':
    all_images = glob.glob('./results/epoch*/*.png')
    print('here with', all_images)

  for j in all_images:
    this_image = cv2.imread(j)
    this_image = this_image[:,223:224+223]
    image_base = j.split(os.path.sep)[-1].split('.')[0] + '.jpg'
    cv2.imwrite(dist_dir + image_base, this_image)  
crop_and_copy('/content/datasets/3d_recon_train/', '*')    

In [ ]:

# os.chdir('/content/Deep3DFaceRecon_pytorch/checkpoints/face_3d_recon/')
# !ls 
# shutil.make_archive('Results', 'zip', './results')

In [ ]:
import shutil
def render_an_image(img_path):

  if not os.path.exists('/content/datasets/test/'):
    os.makedirs('/content/datasets/test/')

  shutil.copy(img_path, '/content/datasets/test/')

  landmarks_to_file('/content/datasets/test/')
  
  try:
    if os.path.exists(glob.glob('./results/epoch*')[0]):
      shutil.rmtree(glob.glob('./results/epoch*')[0])
  except:
    pass
  
  os.chdir('/content/Deep3DFaceRecon_pytorch/')
  os.system('python test.py --name=face_3d_recon --epoch=20 --img_folder=/content/datasets/test/')

  if os.path.exists('/content/datasets/test_recon'):
    shutil.rmtree('/content/datasets/test_recon')

  crop_and_copy('/content/datasets/test_recon/', 'test')

  shutil.rmtree('/content/datasets/test/')


from deepface import DeepFace
def recognize_face(img_path, dataset_path, threshold):
  print('==============\n', img_path, '\n============')

  df_result1 = DeepFace.find(img_path, dataset_path)
  # print(df_result1.head)
  minimum = df_result1[df_result1.columns[1]][0]
  if minimum < threshold:
    print('Input image {0} is recognized as {1}'.format(img_path.split(os.path.sep)[-1], df_result1[df_result1.columns[0]][0].split(os.path.sep)[-1]))
    return df_result1[df_result1.columns[0]][0], df_result1

  else:
    print('Subject is not recognized closest matches are listed below:')
    print(print(df_result1.head))
    return False, df_result1

import random
all_test_imgs = glob.glob(database_test_root + '/*/*')
existing_subject = random.choice(glob.glob(os.path.join(database_test_root, random.choice(select_subjects), '*.*')))
random_img = random.choice(all_test_imgs)
search_path = '/content/datasets/3d_recon_train'
# print(random_img)
render_an_image(existing_subject)
recognition, df_result1 = recognize_face(glob.glob('/content/datasets/test_recon/*')[0], search_path, 0.25)

In [ ]:
df_result1

In [ ]:
df_result1

In [ ]:
# os.chdir('/content/Deep3DFaceRecon_pytorch/')
# !python test.py --name=face_3d_recon --epoch=20 --img_folder='/content/datasets/test/test_1/'